#Zadanie 2 (7 pkt)
Celem zadania jest rozwiązanie zadania optymalizacji przy pomocy algorytmu genetycznego. Należy znaleźć minimum zadanej funkcji:
\begin{equation}
f(x) = 1.5-\exp \left\{-x_1^2-x_2^2\right\}-0.5 \exp \left\{-\left(x_1-1\right)^2-\left(x_2+2\right)^2\right\}
\end{equation}
Należy wykorzystać mechanizmy krzyżowania punktowego, mutacji oraz selekcji turniejowej. Proszę wykorzystać przygotowany szkielet i zaimplementować niezbędne metody. Opracowane oprogramowanie powinno być uniwersalne - wymiar funkcji, której minimum szukamy może być dowolny (mechanizm *args). Punktacja wygląda następująco:

*   Stworzenie obiektu klasy *Chromosome* z polem *array*, które jest wektorem aktywnych i nieaktywnych genów - **0.5 pkt**
*   Implementacja metody *decode*, która dekoduje część chromosomu (określoną przez początek (*lower_bound*) i koniec (*upper_bound*)) do postaci liczby rzeczywistej. *aoi* to zakres wartości zdekodowanej liczby rzeczywistej. Przykład: liczba 135 w postaci binarnej zapisana przy użyciu 8 bitów to 1000 0111, jeśli nasze *aoi* to [0, 1], to 135 sprowadzone do tego zakresu to 0.529. Można skorzystać z funkcji pomocniczej *min_max_norm* - **1 pkt**
*   Implementacja metody *mutation*, która przyjmuje jako argument prawdopodobieństo mutacji i zmienia wartość jedego, losowego genu na przeciwną - **0.5 pkt**
*   Implementacja metody *crossover*, która wykonuje operację krzyżowania jednopunktowego - **1 pkt**
*   Implementacja metody *eval_objective_func*, która dekoduje cały chromosom (otrzymuje się argumenty funkcji) oraz zwraca wartość funkcji celu dla tych argumentów - **1 pkt**
*   Implementacja metody *tournament_selection*, która przeprowadza selekcję turniejową - **1 pkt**
*   Implementacja metody *reproduce*, która generuje nowe pokolenie - z pewnym prawdopodobieństwem przeprowadza krzyżowanie jednopunktowe lub "przerzuca" rodziców do nowego pokolenia - **0.5 pkt**
*   Implementacja metody *run*, która wykonuje cały alorytm genetyczny dla określonej liczby pokoleń. W każdym pokoleniu należy zapisać dane osobnika z najlepszym chromosomem zdekodowane wartości x i y oraz wartość funkcji strat dla tego osobnika - **0.5 pkt**
*   Proszę, podobnie jak miało to miejsce w przypadku metody gradientowej z poprzednich zajęć, wygenerować wykres przy użyciu funkcji *plot_func* (w przypadku innego typu argumentu *trace*, funkcję można zmodyfikować. Wykres powinien przedstawiać funkcję, której minimum poszukujemy oraz punkty odpowiadające najlepszym osobnikom w danych generacjach, których kolor jest coraz jaśniejszy wraz ze zbliżaniem się do minimum. Proszę zapisać swoje wnioski, w szczególności w odniesieniu do metody gradientowej. - **1 pkt**


In [1]:
from typing import Callable, List, Dict, Tuple
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
class Function:

    @staticmethod
    def f(x1, x2):
        return 1.5 - np.exp(-(x1 ** 2) - (x2 ** 2)) - 0.5 * np.exp(-((x1 - 1) ** 2) - ((x2 + 2) ** 2))

In [2]:
def min_max_norm(val, min_val, max_val, new_min, new_max):
  return (val - min_val) * (new_max - new_min) / (max_val - min_val) + new_min

In [3]:
class Chromosome:
  def __init__(self, length, array=None): #if array is None it should be initialized with random binary vector
    """
    Initialize Chromosome with a binary gene array.
    If no array is provided, it generates a random binary vector of given length.
    """
    self.length = length
    self.array = array if array is not None else [random.randint(0, 1) for _ in range(length)]

  def decode(self, lower_bound, upper_bound, aoi):
    """
    Decode binary vector to a real number within the specified range

    @params
    - lower_bound: Starting index of the binary segment.
    - upper_bound: Ending index of the binary segment.
    - aoi: An two element array representing the range of the decoded real number.

    @returns
    - A real value corresponding to the binary segment, normalized withing specified range.
    """

    bin_vec = self.array[lower_bound:upper_bound + 1]
    dec_val = sum(val * (2 ** idx) for idx, val in enumerate(reversed(bin_vec)))

    min_val, max_val = 0, (2 ** len(bin_vec)) - 1
    return min_max_norm(dec_val, min_val, max_val, *aoi)

  def mutation(self, probability):
    """
    Mutate the gene by flipping the bit with the given probability.

    @params
    - probability: Probability of flipping the bit of the gene.
    """

    if random.random() < probability:
      idx = random.randint(0, len(self.array) - 1)
      self.array[idx] = 0 if self.array[idx] == 1 else 1

  def crossover(self, other):
    """
    Perform one-point crossover with another Chromosome object.

    @params
    - other: Other Chromosome object to crossover with.

    @returns
    - Two Chromosome objects resulting from the crossover.
    """

    assert isinstance(other, Chromosome), "You can only crossover with other Chromosome object."

    rand_idx = random.randint(0, len(self.array) - 1)

    first_child_arr = self.array[:rand_idx] + other.array[rand_idx:]
    second_child_arr = other.array[:rand_idx:] + self.array[rand_idx:]

    return Chromosome(self.length, first_child_arr), Chromosome(self.length, second_child_arr)

In [4]:
class GeneticAlgorithm:
  def __init__(self,
               chromosome_length: int,
               obj_func_num_args: int,
               objective_function: Callable[..., float],
               aoi: List[float],
               population_size: int = 1000,
               tournament_size: int = 2,
               mutation_probability: float = 0.05,
               crossover_probability: float = 0.8,
               num_steps: int = 30
               ):
    
    assert chromosome_length % obj_func_num_args == 0, "Number of bits for each argument should be equal" #sprawdzenie ze mamy 16elem chromosom

    self.chromosome_lengths = chromosome_length
    self.obj_func_num_args = obj_func_num_args

    self.bits_per_arg = int(chromosome_length / obj_func_num_args)

    self.objective_function = objective_function
    self.aoi = aoi
    self.tournament_size = tournament_size
    self.mutation_probability = mutation_probability
    self.crossover_probability = crossover_probability
    self.num_steps = num_steps

    # populacje robimy jako listę chromosomów (np. 500 incjancji klasy chromosom losowo)
    self.population = [Chromosome(self.chromosome_lengths) for _ in range(self.population_size)]

    # rodzicow wybieramy w ten sposob ze robimy np 500 razy selekcje turniejowa
    # wybieramy iles np. 6 losowo osobnikow, kodujemy argumenty,
    # wrzucamy argumenty do tej funkcji i zwracamy
    # wybieramy z selekcji tego osobnika ktory zwrocil nam najnizsza wartosc tej funkcji
    # z tego mamy potencjalnych rodzicow

  def eval_objective_func(self, chromosome) -> Dict[Tuple[float, ...], float]:
    assert isinstance(chromosome, Chromosome), "You can only pass Chromosome object."
    # zdekodowanie dwóch argumentów, dzielimy chromosom na pierwsze elementy i drugie elementy (np. dla 16elem 8-8)
    # i wrzucamy do objective function
    # funkcja powinna zwracać dictionary, która ma {argumenty zdekodowane: wartość argumentów}

    decoded_args = []
    for i in range(self.obj_func_num_args):
      start_idx = i * self.bits_per_arg
      end_idx = (i + 1) * self.bits_per_arg - 1

      decoded_args.append(
        chromosome.decode(start_idx, end_idx, self.aoi)
      )

    value = self.objective_function(*decoded_args)

    return {tuple(decoded_args): value}

  def tournament_selection(self):
    # jak na caly wynik optymalizacji wplywa rozmiar turnieju
    # bierzemy iles osobnikow i sprawdzamy ktory z nich najlepiej spelnia funkcje celu
    pass

  def reproduce(self, parents):
    # mamy potencjalnych rodzicow i tworzymy nowa populacje
    # while'em tworzymy taka populacje ktora bedzie miala taki size jak poprzednia

    # bierzemy jakas pare z rodzicow i krzyzujemy ich, albo przechodza do kolejnej populacji (reproduciton probability)
    # jesli w tym kroku ich krzyzujemy tworzymy z nich dwojke dzieci
    # jesli nie krzyzujemy to do kolejnego pokolenia przechodza rodzice a nei dzieci
    pass

  def plot_func(self, trace):
    X = np.arange(-2, 3, 0.05)
    Y = np.arange(-4, 2, 0.05)
    X, Y = np.meshgrid(X, Y)
    Z = 1.5 - np.exp(-X ** (2) - Y ** (2)) - 0.5 * np.exp(-(X - 1) ** (2) - (Y + 2) ** (2))
    plt.figure()
    plt.contour(X, Y, Z, 10)
    cmaps = [[ii / len(trace), 0, 0] for ii in range(len(trace))]
    plt.scatter([x[0] for x in trace], [x[1] for x in trace], c=cmaps)
    plt.show()

  def run(self):
    # nalezy wywolac metody wyzej
    # tworzymy rodzicow z metody turniejowej
    # tworzemy dzieci z reproduce
    # liczymy objective function
    # robimy tak okolo 20 razy

    # zapisujemy to co daje osobnik, nalezy zaznaczyc najlepszego osobnika z danego pokolenia
    pass